<a href="https://www.kaggle.com/code/ahmedelmaamounamin/solubility-prediction?scriptVersionId=214306469" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div align="center">
  <h1 style="text-align: center; padding: 10px;">Solubility Prediction</h1>
</div>

## Introduction

Solubility is a crucial property of organic compounds in the field of chemistry. It refers to the ability of a compound to dissolve in a solvent, typically water. Understanding solubility is essential because it impacts various aspects of chemical research and development, such as drug formulation, environmental impact assessment, and material science.

## Data Source

The data for this project was collected from the ChEMBL website, a prominent repository of bioactive molecules. The dataset used here contains information on organic compounds, including their molecular properties and solubility. As of the last update in January 2023, the dataset comprises an impressive 2,354,965 compounds.

## Data Preparation

The dataset underwent a rigorous cleaning process to ensure its reliability and usability. This cleaning process involved several steps, such as:

- Handling missing values: NaN values were removed or imputed.
- Data type conversion: Certain columns were converted to the appropriate numeric format.
- Removal of duplicates: Duplicate entries were identified and eliminated.

These cleaning steps were essential to prepare the data for accurate and meaningful solubility predictions.

## AlogP and Its Significance

One of the key molecular properties used in solubility prediction is AlogP, or the logarithm of the partition coefficient. AlogP measures the lipophilicity of a compound, indicating how well it can partition between an organic solvent and water. 

In simpler terms, AlogP helps us understand whether a compound prefers to dissolve in a non-polar environment (like oil) or a polar environment (like water). Compounds with higher AlogP values are typically less soluble in water, while those with lower values are more water-soluble.

## Model Training

A linear regression model was trained using the prepared data. The model was trained with the following specifications:

- Number of training epochs: 100,000
- Learning rate: 0.00001
- Evaluation metric: Mean squared error (L2 loss)

Throughout the training process, the model's performance improved gradually. As an illustration, at Epoch 90000, the training L2 loss was observed to be 0.605. This indicates that the model was converging toward a lower loss value, suggesting improved predictive capabilities.


## Predicting AlogP

This project includes the functionality to predict AlogP for a given set of molecular properties. Users can input the feature values for a molecule, and the trained model will provide the predicted AlogP value.

## Interpretation

Interpreting the AlogP prediction is essential. Here's a simple guideline:

- Positive AlogP: Indicates a compound is more hydrophobic and has a greater affinity for the organic phase.
- Negative AlogP: Suggests a compound is more hydrophilic and prefers the aqueous phase.

## Conclusion

Understanding and predicting solubility, especially in the context of AlogP, is vital for various scientific and industrial applications. This project utilizes machine learning techniques to make accurate predictions about the solubility of organic compounds, contributing to advancements in drug discovery, environmental science, and material design.

The combination of data cleaning, feature selection, and model training demonstrates a structured approach to tackling real-world challenges in chemistry and data science.


In [1]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Load the dataset using pandas with the specified delimiter
df = pd.read_csv("/kaggle/input/most-drugs/All Drugs.csv", delimiter=';', on_bad_lines='skip')

# Check if the file is loaded successfully
if not df.empty:
    print(f"Dataset loaded successfully.")
else:
    print(f"Failed to load the dataset.")


/tmp/ipykernel_20/4187779942.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/input/most-drugs/All Drugs.csv", delimiter=';', on_bad_lines='skip')


Dataset loaded successfully.


In [3]:
# Check the column names in the DataFrame
print(df.columns)


Index(['ChEMBL ID', 'Name', 'Synonyms', 'Type', 'Max Phase',
       'Molecular Weight', 'Targets', 'Bioactivities', 'AlogP',
       'Polar Surface Area', 'HBA', 'HBD', '#RO5 Violations',
       '#Rotatable Bonds', 'Passes Ro3', 'QED Weighted', 'CX Acidic pKa',
       'CX Basic pKa', 'CX LogP', 'CX LogD', 'Aromatic Rings',
       'Structure Type', 'Inorganic Flag', 'Heavy Atoms', 'HBA (Lipinski)',
       'HBD (Lipinski)', '#RO5 Violations (Lipinski)',
       'Molecular Weight (Monoisotopic)', 'Np Likeness Score',
       'Molecular Species', 'Molecular Formula', 'Smiles', 'Inchi Key'],
      dtype='object')


In [4]:
# Filter the DataFrame to include only rows where Type is "Small molecule"
small_molecules_df = df[df['Type'] == 'Small molecule']

# Check if any rows match the filter condition
if not small_molecules_df.empty:
    print(f"Filtered DataFrame contains {len(small_molecules_df)} rows with Type 'Small molecule'.")
else:
    print("No rows match the filter condition.")

# Save the filtered DataFrame as a new CSV file called "small_molecules.csv"
small_molecules_df.to_csv("/kaggle/working/small_molecules.csv", index=False)

print("Filtered DataFrame saved as 'small_molecules.csv'.")


Filtered DataFrame contains 1920599 rows with Type 'Small molecule'.
Filtered DataFrame saved as 'small_molecules.csv'.


In [5]:
# Read the small_molecules.csv file
small_molecules_df = pd.read_csv("/kaggle/working/small_molecules.csv")

# List of columns to keep
columns_to_keep = [
    "Molecular Weight",
    "AlogP",
    "Polar Surface Area",
    "HBD",
    "HBA",
    "#Rotatable Bonds",
    "Aromatic Rings",
    "Heavy Atoms",
]

# Filter the DataFrame to include only the selected columns
filtered_small_molecules_df = small_molecules_df[columns_to_keep]

# Check if any rows match the filter condition
if not filtered_small_molecules_df.empty:
    print(f"Filtered DataFrame contains {len(filtered_small_molecules_df)} rows with selected columns.")
else:
    print("No rows match the filter condition.")

# Save the filtered DataFrame as a new CSV file called "solubility_data.csv"
filtered_small_molecules_df.to_csv("/kaggle/working/solubility_data.csv", index=False)

print("Filtered DataFrame with selected columns saved as 'solubility_data.csv'.")


/tmp/ipykernel_20/2304391296.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  small_molecules_df = pd.read_csv("/kaggle/working/small_molecules.csv")


Filtered DataFrame contains 1920599 rows with selected columns.
Filtered DataFrame with selected columns saved as 'solubility_data.csv'.


In [6]:
#Display the list of columns and the data
data = pd.read_csv('/kaggle/working/solubility_data.csv')
x = data[['Molecular Weight', 'Polar Surface Area', 'HBD', 'HBA', '#Rotatable Bonds', 'Aromatic Rings', 'Heavy Atoms']]
# Check the shape of the feature matrix (X)
x.shape
x

,Molecular Weight,Polar Surface Area,HBD,HBA,#Rotatable Bonds,Aromatic Rings,Heavy Atoms
0,356.28,170.05,6,10,3,2,25
1,439.58,48.95,1,6,13,3,31
2,359.36,84.71,1,5,5,2,26
3,395.50,53.09,0,5,7,2,29
4,217.22,55.40,1,3,2,1,16
...,...,...,...,...,...,...,...
1920594,402.51,105.55,3,7,6,3,30
1920595,293.41,23.47,1,2,3,2,22
1920596,398.48,77.23,3,3,7,2,29
1920597,443.56,29.95,1,6,6,2,29


In [7]:
# Convert the specified columns to float64
columns_to_convert = ['AlogP', 'Polar Surface Area', 'HBD', 'HBA', '#Rotatable Bonds', 'Aromatic Rings', 'Heavy Atoms']

for column in columns_to_convert:
    data[column] = pd.to_numeric(data[column], errors='coerce')

In [8]:
# Calculate missing values 
data.isnull().sum()

Molecular Weight       2057
AlogP                 33885
Polar Surface Area    33885
HBD                   33885
HBA                   33885
#Rotatable Bonds      33885
Aromatic Rings        33885
Heavy Atoms           33885
dtype: int64

In [9]:
# To remove rows with missing values (NaN)
data.dropna(inplace=True)

# To impute missing values with the mean
data.fillna(data.mean(), inplace=True)

In [10]:
# Double check that no more missing values
data.isnull().sum()

Molecular Weight      0
AlogP                 0
Polar Surface Area    0
HBD                   0
HBA                   0
#Rotatable Bonds      0
Aromatic Rings        0
Heavy Atoms           0
dtype: int64

In [11]:
x = data[['Molecular Weight', 'Polar Surface Area', 'HBD', 'HBA', '#Rotatable Bonds', 'Aromatic Rings', 'Heavy Atoms']]
y = data['AlogP']

N = len(x)
print(N)

# Define a numpy array containing ones and concatenate it with the feature matrix
ones = np.ones(N)
Xp = np.c_[ones, x]

# Reshape the target variable (y) to be a column vector
y = y.values.reshape(-1, 1)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(Xp, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Initialize the weights (coefficients) with random values
w = 2 * np.random.rand(8, 1) - 1  # Shape (8, 1) for 7 features + bias term


# Function to train the model and return the trained model
def train_model(X_train, y_train):
    # Initialize the weights (coefficients) with random values for training data
    w = 2 * np.random.rand(8, 1) - 1  # Shape (8, 1) for 7 features + bias term

    # Define the number of training epochs and the learning rate
    epochs = 100000
    learning_rate = 0.00001

    # Training loop
    for epoch in range(epochs):
        # Calculate the predicted values using the current weights for training data
        y_train_predicted = X_train @ w

        # Calculate the error for training data
        train_error = y_train - y_train_predicted

        # Calculate the gradient of the loss with respect to the weights for training data
        train_gradient = -(1/X_train.shape[0]) * X_train.T @ train_error

        # Update the weights using gradient descent for training data
        w = w - learning_rate * train_gradient

        # Calculate the mean squared error (L2 loss) for training data
        train_L2 = 0.5 * np.mean(train_error**2)

        # Print progress every 10% of the epochs
        if epoch % (epochs / 10) == 0:
            print(f"Epoch {epoch}: Training L2 Loss = {train_L2}")

    # Create a trained Linear Regression model
    reg = LinearRegression()
    reg.coef_ = w[1:].T  # Set the coefficients
    reg.intercept_ = w[0][0]  # Set the intercept

    return reg

# Train the model using your training data
trained_model = train_model(X_train, y_train)

# Save the trained model to use later
joblib.dump(trained_model, 'trained_model.pkl')


1886714
Epoch 0: Training L2 Loss = 85967.7394515273
Epoch 10000: Training L2 Loss = 1.6595135674334751
Epoch 20000: Training L2 Loss = 0.920992119316307
Epoch 30000: Training L2 Loss = 0.7124544687528018
Epoch 40000: Training L2 Loss = 0.6446511335191651
Epoch 50000: Training L2 Loss = 0.6180876290431295
Epoch 60000: Training L2 Loss = 0.6049441395747253
Epoch 70000: Training L2 Loss = 0.5967616781721491
Epoch 80000: Training L2 Loss = 0.5907428310802685
Epoch 90000: Training L2 Loss = 0.5858656129841503


['trained_model.pkl']

# Predict the AlogP of your compound
#### Just remove the comments (''')  and run it

#### If you don't have the parameters for your compound, you can use the following notebook to calculate them using the SMILE code:
[Chemical Property Calculation with RDKit](https://www.kaggle.com/code/ahmedelmaamounamin/chemical-property-calculation-with-rdkit)

In [12]:
'''
# Function to input feature values and predict A log P
def predict_A_log_P(model):
    print("Enter the feature values for the molecule:")
    try:
        Molecular_Weight = float(input("Molecular Weight: "))
        Polar_Surface_Area = float(input("Polar Surface Area: "))
        HBD = int(input("Number of H-Bond Donors: "))
        HBA = int(input("Number of H-Bond Acceptors: "))
        Rotatable_Bonds = int(input("Number of Rotatable Bonds: "))
        Aromatic_Rings = int(input("Number of Aromatic Rings: "))
        Heavy_Atoms = float(input("Number of Heavy Atoms: "))
    except ValueError:
        print("Invalid input. Please enter valid numeric values for all features.")
        return

    # Create a feature vector from user inputs
    feature_vector = np.array([[Molecular_Weight, Polar_Surface_Area, HBD, HBA, Rotatable_Bonds, Aromatic_Rings, Heavy_Atoms]])

    # Predict A log P using the provided model
    predicted_A_log_P = model.predict(feature_vector)

    print(f"Predicted A log P: {predicted_A_log_P[0][0]:.2f} ")

# Call the predict_A_log_P function with your trained model
predict_A_log_P(trained_model)
'''

'\n# Function to input feature values and predict A log P\ndef predict_A_log_P(model):\n    print("Enter the feature values for the molecule:")\n    try:\n        Molecular_Weight = float(input("Molecular Weight: "))\n        Polar_Surface_Area = float(input("Polar Surface Area: "))\n        HBD = int(input("Number of H-Bond Donors: "))\n        HBA = int(input("Number of H-Bond Acceptors: "))\n        Rotatable_Bonds = int(input("Number of Rotatable Bonds: "))\n        Aromatic_Rings = int(input("Number of Aromatic Rings: "))\n        Heavy_Atoms = float(input("Number of Heavy Atoms: "))\n    except ValueError:\n        print("Invalid input. Please enter valid numeric values for all features.")\n        return\n\n    # Create a feature vector from user inputs\n    feature_vector = np.array([[Molecular_Weight, Polar_Surface_Area, HBD, HBA, Rotatable_Bonds, Aromatic_Rings, Heavy_Atoms]])\n\n    # Predict A log P using the provided model\n    predicted_A_log_P = model.predict(feature_vec